# Multilayer perceptron (MLP)
## Regression advanced

In [1]:
import os

import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
%matplotlib inline
sns.set_theme(context='notebook', style='whitegrid')

#### Funktionen zum Ploten der Trainingsergebnisse

In [ ]:
def plot_results(epochs, history, metric='mean_absolute_error'):
    f, axes = plt.subplots(1,2, figsize=(12,4))

    ax = axes[0]
    sns.lineplot(x=range(epochs), y=history.history['loss'], label='Training', ax=ax)
    sns.lineplot(x=range(epochs), y=history.history['val_loss'], label='Validation', ax=ax)
    ax.set_xlabel("Epochs")
    ax.set_ylabel("Loss")

    ax = axes[1]
    sns.lineplot(x=range(epochs), y=history.history[metric], label=f'Training', ax=ax)
    sns.lineplot(x=range(epochs), y=history.history[f'val_{metric}'], label=f'Validation', ax=ax)
    ax.set_xlabel("Epochs")
    ax.set_ylabel(metric)

    f.tight_layout()
    plt.show()

### Daten einlesen

In [ ]:
# adjust to correct path if necessary
df_rental_bikes = pd.read_csv("../Datasets/rental_bikes.csv")

### Überblick über Daten bekommen

In [ ]:
df_rental_bikes.head()

In [ ]:
df_rental_bikes.describe().round(2)

### Daten vorbereiten

#### Wochentag als Merkmal anlegen

In [ ]:
df_rental_bikes['weekday'] = pd.to_datetime(df_rental_bikes.Date, dayfirst=True ).dt.weekday

#### Zielvariable und nicht benötigte Variable entfernen

In [ ]:
df_rental_bikes_features = df_rental_bikes.drop(columns=['Rented Bike Count', 'Date'])

#### Kategorische Variablen in Dummyvariablen umwandeln

In [ ]:
df_rental_bikes_features = pd.get_dummies(df_rental_bikes_features, columns=['Seasons', 'Holiday', 'Functioning Day'], drop_first=True)

In [ ]:
df_rental_bikes_features

### Standardisieren

In [ ]:
scaler = StandardScaler()
min_max_scaler = MinMaxScaler()

In [ ]:
df_rental_bikes_features.loc[:, "Hour": "Solar Radiation (MJ/m2)"] = scaler.fit_transform(df_rental_bikes_features.loc[:, "Hour": "Solar Radiation (MJ/m2)"])
df_rental_bikes_features.loc[:, ["Rainfall(mm)", "Snowfall (cm)"]] = min_max_scaler.fit_transform(df_rental_bikes_features.loc[:, ["Rainfall(mm)", "Snowfall (cm)"]])

In [ ]:
df_rental_bikes_features.loc[:, "Hour": "weekday"]

#### Für spätere Validierung Daten in Train- und Testset aufteilen

In [ ]:
RANDOM_STATE = 42

keras.utils.set_random_seed(RANDOM_STATE)

In [ ]:
X = df_rental_bikes_features.to_numpy()
y = df_rental_bikes['Rented Bike Count'].to_numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=RANDOM_STATE)

# Aufgabe
## Model Setup
Wählen Sie nun die passenden Layer aus. Beginnen Sie am besten mit einem einfachen Modell und erweitern Sie es nach und nach.
Mögliche Layer sind:
- Dense
- Dropout
- BatchNormalization

Weitere mögliche Erweiterungen sind:
- Alternative Wahl des Optimizers. Hier ist eine Liste möglicher Optionen: https://keras.io/api/optimizers/#available-optimizers
- Weight Decay mit `keras.regularizers.l2()`
- Early Stopping Callback mit `keras.EarlyStopping()`

Außerdem können Sie Hyperparamter anpassen:
- Learning Rate des Optimizers
- Activation Functions der Hidden Layer `("tanh", "relu", "elu")`
- Anzahl Neuronen der Layer
- Batch size
- Epochs

In [ ]:
input_dim = X_train.shape[1]

In [ ]:
model = keras.models.Sequential([
    keras.layers.Input(shape=input_dim),
    #################################################
    #
    # code for layers
    #
    #################################################
    keras.layers.Dense(1, activation='linear'),
])

In [ ]:
LR = 1e-2
BATCHSIZE = 64
EPOCHS = 50

In [ ]:
model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=LR),
    loss=keras.losses.MeanSquaredError(),               ## Wird als Metrik für das Netzwerk verwendet um Error zu berechnen
    metrics=keras.losses.MeanAbsoluteError(),           ## Nur für Monitoring, intern wird nichts mit diesem Wert gemacht
)

In [ ]:
model.summary()

In [ ]:
earlystopping = None # Hier ergänzen!

history = model.fit(
    x=X_train, y=y_train,
    batch_size=BATCHSIZE,
    epochs=EPOCHS,
    validation_split=0.2,
    callbacks=[earlystopping]  # Callbacks werden als Liste zu fit() hinzugefuegt
)

In [ ]:
plot_results(len(history.epoch), history)

### Validierung

<img src="../Bilder/rmse.png" alt="Root Mean Squared Error" height="100"/>

Überprüfen Sie im letzten Schritt, wie gut Ihr Modell Vorhersagen machen kann anhand des RMSE (Root Mean Squared Error) und vergleichen Sie es damit den Durchschnitt vorherzusagen. 

In [ ]:
y_pred = model.predict(X_test)

#### Baseline Ergebnis

In [ ]:
y_mean = np.repeat(np.mean(y_train), len(y_test))

rmse_baseline = mean_squared_error(y_test, y_mean, squared=False).round(2)
mae_baseline = mean_absolute_error(y_test, y_mean).round(2)


print(f"RMSE: {rmse_baseline}")
print(f"MAE: {mae_baseline}")

#### Modell Ergebnis

In [ ]:
y_pred = model.predict(X_test)

rmse_test = mean_squared_error(y_test, y_pred, squared=False).round(2)
mae_test = mean_absolute_error(y_test, y_pred).round(2)

print(f"RMSE: {rmse_test}")
print(f"MAE: {mae_test}")